In [14]:
import numpy as np
import numpy as np
from numpy.random import randn

rate, div, vol, spot, strike = 0.05, 0.03, 0.15, 100.0, 120.0

def create_model():
    return np.array([rate, div, vol, spot, strike])

n_paths = 2 ** 20
T = 3.0

In [15]:
default_model = create_model()

## 0. DAL
------------------

In [16]:
# !pip install dal-python -U

from dal import *
import dal
dal.__version__

'0.3.5'

In [17]:
today = Date_(2022, 9, 15)
EvaluationDate_Set(today)

maturity = today.AddDays(int(365 * T))

In [18]:
%%time
event_dates = ["STRIKE", maturity]
events = [f"{strike}", f"call pays MAX(spot() - STRIKE, 0.0)"]

product = Product_New(event_dates, events)
model = BSModelData_New(spot, vol, rate, div)

# only price
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, False)
dict(res)

CPU times: total: 281 ms
Wall time: 42 ms


{'value': 5.201768833617436}

In [19]:
%%time

# price with derivatives
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, True)
dict(res)

CPU times: total: 625 ms
Wall time: 84 ms


{'d_div': -100.50943481175734,
 'd_rate': 84.90412831090498,
 'd_spot': 0.33503144937252466,
 'd_vol': 59.585032759891966,
 'value': 5.201768833617436}

## 1. DAL.JAX
----------------

In [20]:
import jax
import jax.numpy as jnp

In [21]:
def compute_call_price_jax(model, T = 3.0, M=n_paths, key=jax.random.PRNGKey(1)):
    """
    Estimate the price of the call option using Monte Carlo.
    """
    # Set up
    μ, d, σ, S, K = model
    s = jnp.full(M, jnp.log(S))
    Z = jax.random.normal(key, (1, M))
    s = s + (μ - d - 0.5 * σ * σ) * T +  σ * jnp.sqrt(T) * Z[0, :]
    expectation = jnp.mean(jnp.maximum(jnp.exp(s) - K, 0))
    return jnp.exp(-μ * T) * expectation

compute_call_price_jax_jit = jax.jit(compute_call_price_jax)
_ = compute_call_price_jax(default_model).block_until_ready()
_ = compute_call_price_jax_jit(default_model).block_until_ready()

In [22]:
%%time

# only price without jit
compute_call_price_jax(default_model).block_until_ready()

CPU times: total: 109 ms
Wall time: 24 ms


Array(5.198222, dtype=float32)

In [23]:
%%time

# only price with jit
compute_call_price_jax_jit(default_model).block_until_ready()

CPU times: total: 46.9 ms
Wall time: 11 ms


Array(5.198222, dtype=float32)

In [24]:
compute_call_value_and_grad = jax.value_and_grad(compute_call_price_jax)
compute_call_value_and_grad_jit = jax.value_and_grad(jax.jit(compute_call_price_jax))

_ = compute_call_value_and_grad(default_model)[0].block_until_ready()
_ = compute_call_value_and_grad_jit(default_model)[0].block_until_ready()

In [25]:
%%time

# price and derivatives without jit
result = compute_call_value_and_grad(default_model)
result[0].block_until_ready()
result[1].block_until_ready()
result

CPU times: total: 78.1 ms
Wall time: 54 ms


(Array(5.198222, dtype=float32),
 Array([  84.90206   , -100.49673   ,   59.547592  ,    0.3349891 ,
          -0.23583907], dtype=float32))

In [26]:
%%time

# price and derivatives with jit
result = compute_call_value_and_grad_jit(default_model)
result[0].block_until_ready()
result[1].block_until_ready()
result

CPU times: total: 125 ms
Wall time: 19 ms


(Array(5.198222, dtype=float32),
 Array([  84.90206   , -100.49673   ,   59.547592  ,    0.3349891 ,
          -0.23583907], dtype=float32))